In [87]:
! pip install -r requirements.txt --quiet

In [88]:
from dotenv import load_dotenv
from os import environ
from langchain_openai import AzureChatOpenAI
from typing_extensions import TypedDict,Literal
from langgraph.types import Command
from langgraph.graph import MessagesState, StateGraph, START, END
from IPython.display import Image, display
from langgraph.prebuilt import ToolNode
from  user_tools import get_weather_tool
from user_functions import vector_search  
from utils import pretty_print_messages
from langchain_core.messages import HumanMessage,SystemMessage
from langgraph.prebuilt import create_react_agent
from langchain_community.agent_toolkits.openapi.toolkit import OpenAPIToolkit
from langchain_community.utilities.requests import RequestsWrapper
from langchain_community.tools.json.tool import JsonSpec
import requests
import json

load_dotenv(override=True)



members = ["document_search", "weather", "datacenterenergyusage"]

options = members + ["FINISH"]

system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    f" following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH.n\nNote: The 'datacenterenergyusage' worker handles queries related to data center tabular quries on energy, power, or consumption topics."
)

class Router(TypedDict):
    """Worker to route to next. If no workers needed, route to FINISH."""

    next: Literal[*options]


llm = AzureChatOpenAI(
        temperature=0,
        azure_deployment=environ["AZURE_OPENAI_MODEL"],
        api_version=environ["AZURE_OPENAI_API_VERSION"]
    )




In [89]:
response = requests.get("https://api-foundry-lab-ht4kvusmjz8h.azurewebsites.net/openapi.json")

json_spec = JsonSpec(dict_=json.loads(response.text), max_value_length=4000)

In [90]:
headers = {"Authorization": f"Bearer {environ['AZURE_OPENAI_API_KEY']}"}
openai_requests_wrapper = RequestsWrapper(headers=headers)
openapi_toolkit = OpenAPIToolkit.from_llm(llm=llm, json_spec=json_spec,requests_wrapper=openai_requests_wrapper, allow_dangerous_requests=True)
openapi_tools = openapi_toolkit.get_tools()


llm_with_tools = create_react_agent(llm, tools=[get_weather_tool])
llm_with_energy_tools = create_react_agent(llm, tools=openapi_tools)


In [91]:

def document_search(state: MessagesState) -> Command[Literal["supervisor"]]:
    messages = state["messages"]
    last_user_message = next((msg for msg in reversed(messages)), None)
    if not last_user_message:
        return Command(goto="supervisor")

    query = last_user_message.content
    context = vector_search(query)  

    
    messages.append(
    SystemMessage(
        content=(
            f"You are a helpful assistant. Use only the information in the context below to answer the user's question. "
            f"If the context does not contain the answer, respond with \"I don't know.\"\n\nContext:\n{context}"
        )
    )
)

    response = llm.invoke(messages)

    return Command(
        update={
            "messages": [
                HumanMessage(content=response.content, name="document_search")
            ]
        },
        goto="supervisor",
    )


def weather(state: MessagesState) -> Command[Literal["supervisor"]]:
    result = llm_with_tools.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="weather")
            ]
        },
        goto="supervisor",
    )

def datacenterenergyusage(state: MessagesState) -> Command[Literal["supervisor"]]:
    result = llm_with_energy_tools.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="datacenterenergyusage")
            ]
        },
        goto="supervisor",
    )

def supervisor(state: MessagesState) -> Command[Literal[*members, "__end__"]]:
    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"]
    response = llm.with_structured_output(Router).invoke(messages)
    goto = response["next"]
    if goto == "FINISH":
        goto = END

    return Command(goto=goto, update={"next": goto})




In [92]:

builder = StateGraph(MessagesState)
builder.add_node("document_search", document_search)
builder.add_node("weather", weather)
builder.add_node("datacenterenergyusage", datacenterenergyusage)
builder.add_node("supervisor", supervisor)

builder.set_entry_point("supervisor")

graph = builder.compile()


In [93]:
display(Image(graph.get_graph().draw_mermaid_png()))

ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)

In [96]:
for step in graph.stream({"messages": [{"role": "user", "content": "What data centers are in 'critical'"}]}):
   pretty_print_messages(step)

Update from node supervisor:




Update from node datacenterenergyusage:


================================ Human Message =================================
Name: datacenterenergyusage

I couldn't retrieve the exact endpoint or parameters to find data centers with a 'critical' status. Could you clarify if there's a specific API or system you're referring to? Alternatively, I can explore further if you provide more details.


Update from node supervisor:




Update from node datacenterenergyusage:


================================ Human Message =================================
Name: datacenterenergyusage

I couldn't determine the exact query parameters or endpoint to retrieve data centers with a 'critical' status. Could you provide more details or clarify the system/API you're referring to?


Update from node supervisor:




Update from node datacenterenergyusage:


================================ Human Message =================================
Name: datacenterenergyusage

I couldn't 

In [ ]:
for step in graph.stream({"messages": [{"role": "user", "content": "What data centers are in 'critical'"}]}):
   pretty_print_messages(step)

Update from node supervisor:




Update from node datacenterenergyusage:


================================ Human Message =================================
Name: datacenterenergyusage

The power consumption of a data center can vary widely depending on its size, purpose, and efficiency. Here are some general insights:

### 1. **Small Data Centers**
   - **Power Usage**: Small data centers, such as those used by small businesses, may consume anywhere from **10 kW to 1 MW** of power.
   - These are often on-premises facilities with a limited number of servers and cooling systems.

### 2. **Medium-Sized Data Centers**
   - **Power Usage**: Medium-sized data centers typically consume between **1 MW and 10 MW**.
   - These are often used by larger enterprises or colocation providers.

### 3. **Large Hyperscale Data Centers**
   - **Power Usage**: Hyperscale data centers, like those operated by Google, Amazon, or Microsoft, can consume **10 MW to 100 MW or more**.
   - These facilities house